# 뉴스 가져오기
- client id : IamYBKISQmZU5MdHRnsm
- client secret: AEZKqyZhux
-> 네이버 검색 api를 사용하여 해당 기업들의 뉴스를 최신 순으로 100개씩 가져오기
-> 그 후 내가 입력한 날짜와 뉴스 날짜가 맞고, title이나 description에 키워드 리스트에 있는 단어가 있고, avoid keyword list에 있는 단어가 포함되지 않으면 데이터프레임화한다(정확한 뉴스를 가져오기 위함)
- 네이버는 뉴스 검색 시 기사 제목(title)과 본문(description)에 검색 키워드가 포함된 뉴스를 우선적으로 반환

In [30]:

related_keywords  = {
    '정보보안': ['사이버', '해킹', '랜섬웨어', '보안', '위협', '데이터', '보호', '보안 사고', '시스템', '네트워크', '개인정보'],
    '클라우드': ['컴퓨팅', '서버', '데이터', '가상화', '서비스', '온라인', '인터넷', 'IT',  '인프라', '기업', '웹'],
    '제로트러스트': ['보안', '정책', '접근', '제어', '네트워크', '보안', '인증', '데이터', '기업', '인증', '비밀번호', '권한', '관리'],
    '정보유출사고': ['데이터 유출', '보안 사고', '개인정보', '기업 보안', '고객 정보', '정보 보호', '정보 유출', '사이버 공격', '데이터 침해'],
    'NEIS': ['교육', '시스템', '학교', '관리', '행정', '학생', '정보', '교사', '업무', '교육청', '전자', '문서', '공공', 'IT', '디지털', '행정', '나이스'],
    '블록체인': ['디지털', '자산', '거래', '시스템', '보안', '기술', '데이터', '무결성', '탈중앙화', '네트워크', '금융', '기술', '온라인', '거래', '암호화'],
    'STO': ['증권', '투자', '자산', '관리', '금융', '시장', '거래', '블록체인', '금융', '기업', '투자', '디지털', '금융', '경제'],
    'AI': ['인공지능', '기술', '데이터', '분석', '컴퓨터', '자동화', '빅데이터', '딥러닝', '소프트웨어', '산업', '혁신'],
    '망분리': ['보안', '네트워크', '기업 보안', '인터넷', '분리', '데이터', '보호', '네트워크', '관리', '정보', '보호', '보안', '인프라', '기업', 'IT'],
    'IT업계': ['소프트웨어', '하드웨어', '기업', '스타트업', '기술 산업', '디지털', '혁신', '시장', '비즈니스'],
    '보안업계': ['보안', '기술', '보안', '서비스', '위협', '관리', '사이버', '공격', '정보', '보호', '보안', '솔루션', '기업', '보안', '데이터', '관리']
}

unique_keywords = list(set(sum(related_keywords.values(), [])))
print(unique_keywords)

['기업 보안', '행정', '업무', '자동화', '산업', '데이터', '거래', '네트워크', '딥러닝', '보호', '기업', '웹', '관리', '나이스', '문서', '금융', '교사', '접근', '데이터 침해', '랜섬웨어', '투자', '인공지능', '하드웨어', '분리', '학교', '보안 사고', '학생', '사이버 공격', '인터넷', '해킹', '시장', '자산', 'IT', '교육', '사이버', '탈중앙화', '정보 유출', '권한', '솔루션', '컴퓨터', '교육청', '시스템', '위협', '데이터 유출', '블록체인', '정보', '혁신', '스타트업', '공공', '증권', '기술 산업', '비즈니스', '정책', '비밀번호', '정보 보호', '컴퓨팅', '소프트웨어', '고객 정보', '인증', '분석', '공격', '디지털', '인프라', '서비스', '경제', '기술', '보안', '전자', '무결성', '서버', '암호화', '온라인', '제어', '빅데이터', '개인정보', '가상화']


In [31]:
target_date = input("날짜 입력(형식: 20250131): ")

날짜 입력(형식: 20250131): 20250203


In [32]:
import urllib.request
import datetime
import json
import pandas as pd
from urllib.parse import quote
import re

# pd.set_option('display.max_rows', None)

# 네이버 API 인증 정보
client_id = "IamYBKISQmZU5MdHRnsm"
client_secret = "AEZKqyZhux"

# 검색할 기업 리스트 (기업명 -> 종목 코드 매핑)
# related_keywords = {
#     "SGA": "049470", "SGA솔루션즈":"184230", "voiceye": "0", "안랩":"053800", "시큐브":"131090", "윈스":"136540", "이글루":"067920",
#     "한컴위드":"054920", "네오리진": "094860", "케이사인":"192250", "이스트소프트":"047560", "라온시큐어": "042510","파수":"150900",
#     "한국정보인증": "053300", "지란지교시큐리티":"208350","수산아이앤티":"050960", "지니언스": "263860", "드림시큐리티": "203650",
#     "모니터랩":"434480", "파이오링크":"170790", "시큐센":"232830", "시큐레터": "418250",
#     "다우기술":"023590", "다우데이타":"032190", "한글과컴퓨터": "030520", "MDS테크":"086960", "아이티센":"124500", "콤텍시스템": "031820", "소프트센":"032680",
#     "대신정보통신":"020180", "쌍용정보통신":"010280", "에스넷": "038680"
# }

related_keywords = {
    '정보보안': ['사이버', '해킹', '랜섬웨어', '보안', '위협', '데이터', '보호', '보안 사고', '시스템', '네트워크', '개인정보', '정보보안'],
    '클라우드': ['컴퓨팅', '서버', '데이터', '가상화', '서비스', '온라인', '인터넷', 'IT',  '인프라', '기업', '웹'],
    '제로트러스트': ['보안', '정책', '접근', '제어', '네트워크', '보안', '인증', '데이터', '기업', '인증', '비밀번호', '권한', '관리'],
    '정보유출사고': ['데이터 유출', '보안 사고', '개인정보', '기업 보안', '고객 정보', '정보 보호', '정보 유출', '사이버 공격', '데이터 침해'],
    '나이스': ['교육', '시스템', '학교', '관리', '행정', '학생', '정보', '교사', '업무', '교육청', '전자', '문서', '공공', 'IT', '디지털', '행정', 'NEIS', '나이스'],
    '블록체인': ['디지털', '자산', '거래', '시스템', '보안', '기술', '데이터', '무결성', '탈중앙화', '네트워크', '금융', '기술', '온라인', '거래', '암호화', '블록체인'],
    'STO': ['증권', '투자', '자산', '관리', '금융', '시장', '거래', '블록체인', '금융', '기업', '투자', '디지털', '금융', '경제', 'STO'],
    'AI': ['인공지능', '기술', '데이터', '분석', '컴퓨터', '자동화', '빅데이터', '딥러닝', '소프트웨어', '산업', '혁신', 'AI'],
    '망분리': ['보안', '네트워크', '기업 보안', '인터넷', '분리', '데이터', '보호', '네트워크', '관리', '정보', '보호', '보안', '인프라', '기업', 'IT', '망분리'],
    'IT업계': ['소프트웨어', '하드웨어', '기업', '스타트업', '기술 산업', '디지털', '혁신', '시장', '비즈니스', 'IT'],
    '보안업계': ['보안', '기술', '보안', '서비스', '위협', '관리', '사이버', '공격', '정보', '보호', '보안', '솔루션', '기업', '보안', '데이터', '관리']
}

ECON_IT_KEYWORDS = list(set(sum(related_keywords.values(), [])))

# 포함할 키워드 리스트 (경제/IT 관련 키워드)


# 제외할 키워드 리스트 (불필요한 뉴스 필터링)
EXCLUDE_KEYWORDS = ["연예", "스포츠", "날씨", "범죄", "게임", "드라마", "영화", "NBA", "축제", "추위", "호텔", "얼음"]

# 네이버 API 요청 함수
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            return response.read().decode('utf-8')
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# 네이버 뉴스 검색 API 호출 함수 (최대 100개만 가져오기)
def getNaverSearch(keyword):
    base = "https://openapi.naver.com/v1/search/news.json"
    parameters = f"?query={quote(keyword)}&start=1&display=100&sort=date"  # 최신순 정렬
    url = base + parameters

    response = getRequestUrl(url)
    if response is None:
        return None
    else:
        return json.loads(response)

# 뉴스 데이터 필터링 함수 (경제/IT 키워드 포함 여부 + 특정 키워드 제외 여부)
def filterEconomyITNews(post,company):
    """
    기사 제목이나 설명에 ECON_IT_KEYWORDS(경제/IT 키워드)가 포함되고,
    EXCLUDE_KEYWORDS(제외할 키워드)가 포함되지 않은 경우만 True 반환
    """
    title = post['title']
    description = post['description']

    # 포함할 키워드 체크 (하나라도 포함되면 OK)
    if not any(keyword in title or keyword in description for keyword in ECON_IT_KEYWORDS):
        return False  # 경제/IT 관련 기사가 아님

    if not (company in title or company in description):
        return False  # description, title에 해당 기업의 이름이 들어가야

    # 제외할 키워드 체크 (하나라도 포함되면 False)
    if any(keyword in title or keyword in description for keyword in EXCLUDE_KEYWORDS):
        return False  # 제외할 키워드가 포함된 기사이므로 필터링

    return True  # 경제/IT 관련 기사이면서, 제외할 키워드가 포함되지 않은 경우

# 검색 실행 함수 (기업 리스트 전체 순회)
def main():
    # target_date = input("📅 검색할 날짜를 입력하세요 (YYYYMMDD): ").strip()

    all_news = []  # 모든 기업의 뉴스를 저장할 리스트
    total_cnt = 0  # 전체 뉴스 개수 카운트

    for company in related_keywords.keys():
        jsonResult = []
        cnt = 0

        # API 요청
        jsonResponse = getNaverSearch(company)

        if jsonResponse is None or 'items' not in jsonResponse:
            print(f"❌ {company} 뉴스 검색 결과가 없습니다.")
            continue

        total = jsonResponse['total']
        print(f"🔎 {company} 전체 검색 결과: {total}건 (최대 100개 검색)")

        # 뉴스 가져오기 및 날짜 + 키워드 필터링
        for post in jsonResponse['items']:
            # 날짜 변환 (YYYYMMDD)
            pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
            pDate = pDate.strftime('%Y%m%d')

            # 입력한 날짜와 일치하고, 경제/IT 키워드 포함 & 불필요한 뉴스 제외
            if pDate == target_date and filterEconomyITNews(post, company):
                cnt += 1
                jsonResult.append({
                    'cnt': cnt,
                    'company': company,  # 기업명 추가
                    'title': post['title'],
                    'description': post['description'],
                    'org_link': post['originallink'],
                    'link': post['link'],
                    'pDate': pDate
                })

        # 필터링된 뉴스만 리스트에 추가
        all_news.extend(jsonResult)
        total_cnt += len(jsonResult)

    # JSON 저장
    filename = f"CyberSecurity_News_{target_date}.json"
    with open(filename, 'w', encoding='utf8') as outfile:
        json.dump(all_news, outfile, indent=4, sort_keys=True, ensure_ascii=False)

    print(f"✅ 총 가져온 데이터: {total_cnt}건")
    print(f"📂 저장 완료: {filename}")

    # 결과를 DataFrame으로 변환하여 CSV 저장
    df = pd.DataFrame(all_news)
    if not df.empty:
        df.to_csv(f"CyberSecurity_News_{target_date}.csv", encoding='utf-8-sig', index=False)
        print(f"📂 CSV 저장 완료: CyberSecurity_News_{target_date}.csv")
        display(df)
    else:
        print("❌ 해당 날짜에 대한 경제/IT 뉴스가 없습니다.")

if __name__ == '__main__':
    main()


🔎 정보보안 전체 검색 결과: 338116건 (최대 100개 검색)
🔎 클라우드 전체 검색 결과: 1224578건 (최대 100개 검색)
🔎 제로트러스트 전체 검색 결과: 17857건 (최대 100개 검색)
🔎 정보유출사고 전체 검색 결과: 149000건 (최대 100개 검색)
🔎 나이스 전체 검색 결과: 198932건 (최대 100개 검색)
🔎 블록체인 전체 검색 결과: 967079건 (최대 100개 검색)
🔎 STO 전체 검색 결과: 24301건 (최대 100개 검색)
🔎 AI 전체 검색 결과: 3829582건 (최대 100개 검색)
🔎 망분리 전체 검색 결과: 18228건 (최대 100개 검색)
🔎 IT업계 전체 검색 결과: 186161건 (최대 100개 검색)
🔎 보안업계 전체 검색 결과: 315362건 (최대 100개 검색)
✅ 총 가져온 데이터: 76건
📂 저장 완료: CyberSecurity_News_20250203.json
📂 CSV 저장 완료: CyberSecurity_News_20250203.csv


,cnt,company,title,description,org_link,link,pDate
0,1,정보보안,"동서발전, ‘<b>정보</b>보호 국제 표준인증’ 10년 연속 유지",정보보호인증(ISO27001)은 정보보호 및 관리 분야의 가장 권위 있는 국제인증으...,http://www.lawissue.co.kr/view.php?ud=20250203...,http://www.lawissue.co.kr/view.php?ud=20250203...,20250203
1,2,정보보안,"동서발전, ISO '<b>정보</b>보호'·'개인<b>정보</b> 보호' 표준 사후...","ISO27001은 정보보호 및 관리 분야 국제인증으로 정보보호 정책, 물리적 보안 ...",http://www.seoulwire.com/news/articleView.html...,http://www.seoulwire.com/news/articleView.html...,20250203
2,3,정보보안,"동서발전,'<b>정보</b>보호 국제표준 인증' 10년 연속 유지",정보 보호인증(ISO27001)은 정보보호 및 관리 분야의 가장 권위 있는 국제인증...,https://www.ulsanpress.net/news/articleView.ht...,https://www.ulsanpress.net/news/articleView.ht...,20250203
3,4,정보보안,동서발전 '<b>정보</b>보호 국제 표준인증' 10년 연속 유지,"정보보호인증은 정보보호 및 관리 분야의 가장 권위 있는 국제인증으로 정보보호 정책,...",https://www.newscj.com/news/articleView.html?i...,https://www.newscj.com/news/articleView.html?i...,20250203
4,5,정보보안,"동서발전, 10년 연속 '개인·<b>정보</b>보호 국제 표준인증'",정보보호인증(ISO27001)은 정보보호 및 관리 분야의 가장 권위 있는 국제인증으...,https://www.iusm.co.kr/news/articleView.html?i...,https://www.iusm.co.kr/news/articleView.html?i...,20250203
...,...,...,...,...,...,...,...
71,14,IT업계,이재명 &quot;반도체 전문가들 동의할 경우 52시간 예외 허용 거절하기 어려...,"또한 반도체 연구개발 분야에 예외 조항을 두면 향후 생산이나 협력업체, <b>IT업...",https://www.newspim.com/news/view/20250203000712,https://www.newspim.com/news/view/20250203000712,20250203
72,15,IT업계,샘 올트먼 한국서 정신아 만난다…오픈AI·카카오 깜짝 동맹 발표하나,올트먼 CEO가 한국시장에 큰 관심을 두고 있는 만큼 진출이 유력하다는 것이 <b>...,https://www.mk.co.kr/article/11231120,https://n.news.naver.com/mnews/article/009/000...,20250203
73,1,보안업계,'유퀴즈' 나왔던 거물급 해커…&quot;이젠 아시아 대표하는 韓 <b>보안</b>...,◆신한은행 글로벌 법인·삼성그룹 통합 금융서비스 '모니모' 등에 앱수트 공급 박찬암...,https://www.widedaily.com/news/articleView.htm...,https://www.widedaily.com/news/articleView.htm...,20250203
74,2,보안업계,"[기업INSIDE] SGA솔루션즈, 제로트러스트로 차세대 <b>보안</b> 선도",이에 <b>보안업계</b>가 새로운 전환점을 맞이하고 있는데요. 제로트러스트 보안을...,https://news.mtn.co.kr/news-detail/20250203164...,https://news.mtn.co.kr/news-detail/20250203164...,20250203


In [33]:
df = pd.read_csv(f"/content/CyberSecurity_News_{target_date}.csv")

# 우선적으로 유지할 조건 설정 (title에 "업계" 또는 "동향" 포함)
priority_df = df[df['title'].str.contains('업계|동향', na=False, regex=True)]


# 일반 중복 제거 과정
df_unique = df.drop_duplicates(subset=["title"], keep="first")

# 기존 df_unique에서 "업계" 또는 "동향"이 포함되지 않은 행만 남기기
df_non_priority = df_unique[~df_unique['title'].str.contains('업계|동향', na=False, regex=True)]

# 두 개의 데이터프레임을 결합 (우선 필터된 데이터 먼저 배치)
df_priority_first = pd.concat([priority_df, df_non_priority]).drop_duplicates(subset=["title"], keep="first")


# 회사별 중복 제거 (첫 번째 등장하는 회사만 유지)
df_filtered = df_priority_first[~df_priority_first.duplicated(subset=['company'], keep='first')]

# 최종 데이터 출력
display(df_filtered)


,cnt,company,title,description,org_link,link,pDate
55,7,망분리,[주간 클라우드 동향/2월①] MS 성장엔진 클라우드 4분기 실적에 쏟아진...,◆국가망보안체계 예고편 나왔다…'N²SF 가이드라인' 초안 공개=공공기관이 <b>망...,https://www.ddaily.co.kr/page/view/20250203094...,https://n.news.naver.com/mnews/article/138/000...,20250203
59,2,IT업계,"[뉴공 아카이브]박태웅 녹서포럼 의장, 유시민 작가: 美 <b>IT업계</b> 뒤흔...","아래 인터뷰는 2025년 2월 03일자, &quot;김어준의 겸손은힘들다 뉴스공장&...",http://www.ddanzi.com/834253806,http://www.ddanzi.com/834253806,20250203
0,1,정보보안,"동서발전, ‘<b>정보</b>보호 국제 표준인증’ 10년 연속 유지",정보보호인증(ISO27001)은 정보보호 및 관리 분야의 가장 권위 있는 국제인증으...,http://www.lawissue.co.kr/view.php?ud=20250203...,http://www.lawissue.co.kr/view.php?ud=20250203...,20250203
28,1,제로트러스트,"[기업INSIDE] SGA솔루션즈, <b>제로트러스트</b>로 차세대 보안 선도",<b>제로트러스트</b> 보안을 앞세워 주목받는 국내 보안기업 SGA솔루션를 김경문...,https://news.mtn.co.kr/news-detail/20250203164...,https://news.mtn.co.kr/news-detail/20250203164...,20250203
31,1,STO,"[마켓인]부동산 조각투자 스타트업 원컵, 싱가포르 진출 본격화",조각투자 플랫폼 원컵프로 운영사 원컵이 싱가포르 RWA(실물연계자산) 토큰 발행을 ...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250203
73,1,보안업계,'유퀴즈' 나왔던 거물급 해커…&quot;이젠 아시아 대표하는 韓 <b>보안</b>...,◆신한은행 글로벌 법인·삼성그룹 통합 금융서비스 '모니모' 등에 앱수트 공급 박찬암...,https://www.widedaily.com/news/articleView.htm...,https://www.widedaily.com/news/articleView.htm...,20250203


In [34]:
# df = pd.read_csv(f"/content/CyberSecurity_News_{target_date}.csv")

# # display(df)
# df_unique = df.drop_duplicates(subset=["title"], keep="first")
# df_filtered = df_unique[~df_unique.duplicated(subset=['company'], keep='first')]
# display(df_filtered)

##  기업 데이터(row)가 없다면 뉴스 없음으로 넣어주기

In [35]:
# 데이터프레임에 없는 기업 목록 찾기
missing_companies = [company for company in related_keywords.keys() if company not in df_filtered["company"].values]

# 없는 기업들을 추가할 데이터프레임 생성
missing_data = pd.DataFrame({
    "cnt": 1,
    "company": missing_companies,
    "title": "뉴스 없음",
    "description": "뉴스 없음",
    "org_link": "뉴스 없음",
    "link": "뉴스 없음",
    "pDate": int(target_date)  # 기본 날짜 설정 (필요 시 수정 가능)
})

# 기존 데이터프레임과 합치기
df_filtered_all_comp = pd.concat([df_filtered, missing_data], ignore_index=True)
display(df_filtered_all_comp)

,cnt,company,title,description,org_link,link,pDate
0,7,망분리,[주간 클라우드 동향/2월①] MS 성장엔진 클라우드 4분기 실적에 쏟아진...,◆국가망보안체계 예고편 나왔다…'N²SF 가이드라인' 초안 공개=공공기관이 <b>망...,https://www.ddaily.co.kr/page/view/20250203094...,https://n.news.naver.com/mnews/article/138/000...,20250203
1,2,IT업계,"[뉴공 아카이브]박태웅 녹서포럼 의장, 유시민 작가: 美 <b>IT업계</b> 뒤흔...","아래 인터뷰는 2025년 2월 03일자, &quot;김어준의 겸손은힘들다 뉴스공장&...",http://www.ddanzi.com/834253806,http://www.ddanzi.com/834253806,20250203
2,1,정보보안,"동서발전, ‘<b>정보</b>보호 국제 표준인증’ 10년 연속 유지",정보보호인증(ISO27001)은 정보보호 및 관리 분야의 가장 권위 있는 국제인증으...,http://www.lawissue.co.kr/view.php?ud=20250203...,http://www.lawissue.co.kr/view.php?ud=20250203...,20250203
3,1,제로트러스트,"[기업INSIDE] SGA솔루션즈, <b>제로트러스트</b>로 차세대 보안 선도",<b>제로트러스트</b> 보안을 앞세워 주목받는 국내 보안기업 SGA솔루션를 김경문...,https://news.mtn.co.kr/news-detail/20250203164...,https://news.mtn.co.kr/news-detail/20250203164...,20250203
4,1,STO,"[마켓인]부동산 조각투자 스타트업 원컵, 싱가포르 진출 본격화",조각투자 플랫폼 원컵프로 운영사 원컵이 싱가포르 RWA(실물연계자산) 토큰 발행을 ...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250203
5,1,보안업계,'유퀴즈' 나왔던 거물급 해커…&quot;이젠 아시아 대표하는 韓 <b>보안</b>...,◆신한은행 글로벌 법인·삼성그룹 통합 금융서비스 '모니모' 등에 앱수트 공급 박찬암...,https://www.widedaily.com/news/articleView.htm...,https://www.widedaily.com/news/articleView.htm...,20250203
6,1,클라우드,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203
7,1,정보유출사고,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203
8,1,나이스,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203
9,1,블록체인,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203


In [36]:
# 🔹 기업명을 related_keywords.keys() 순서대로 정렬
df_filtered_all_comp["company"] = pd.Categorical(df_filtered_all_comp["company"], categories=related_keywords.keys(), ordered=True)
df_filtered_all_comp = df_filtered_all_comp.sort_values("company").reset_index(drop=True)
display(df_filtered_all_comp)

,cnt,company,title,description,org_link,link,pDate
0,1,정보보안,"동서발전, ‘<b>정보</b>보호 국제 표준인증’ 10년 연속 유지",정보보호인증(ISO27001)은 정보보호 및 관리 분야의 가장 권위 있는 국제인증으...,http://www.lawissue.co.kr/view.php?ud=20250203...,http://www.lawissue.co.kr/view.php?ud=20250203...,20250203
1,1,클라우드,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203
2,1,제로트러스트,"[기업INSIDE] SGA솔루션즈, <b>제로트러스트</b>로 차세대 보안 선도",<b>제로트러스트</b> 보안을 앞세워 주목받는 국내 보안기업 SGA솔루션를 김경문...,https://news.mtn.co.kr/news-detail/20250203164...,https://news.mtn.co.kr/news-detail/20250203164...,20250203
3,1,정보유출사고,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203
4,1,나이스,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203
5,1,블록체인,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203
6,1,STO,"[마켓인]부동산 조각투자 스타트업 원컵, 싱가포르 진출 본격화",조각투자 플랫폼 원컵프로 운영사 원컵이 싱가포르 RWA(실물연계자산) 토큰 발행을 ...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250203
7,1,AI,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203
8,7,망분리,[주간 클라우드 동향/2월①] MS 성장엔진 클라우드 4분기 실적에 쏟아진...,◆국가망보안체계 예고편 나왔다…'N²SF 가이드라인' 초안 공개=공공기관이 <b>망...,https://www.ddaily.co.kr/page/view/20250203094...,https://n.news.naver.com/mnews/article/138/000...,20250203
9,2,IT업계,"[뉴공 아카이브]박태웅 녹서포럼 의장, 유시민 작가: 美 <b>IT업계</b> 뒤흔...","아래 인터뷰는 2025년 2월 03일자, &quot;김어준의 겸손은힘들다 뉴스공장&...",http://www.ddanzi.com/834253806,http://www.ddanzi.com/834253806,20250203


In [37]:
!pip install newspaper3k
!pip install --upgrade lxml newspaper3k lxml_html_clean

In [38]:
!pip install fuzzywuzzy

In [39]:
import pandas as pd
import re
from newspaper import Article
from fuzzywuzzy import fuzz

# ✅ `description` 텍스트 정리 함수
def clean_description(text):
    text = re.sub(r"<\/?b>", "", text)  # <b>, </b> 태그 제거
    text = text.replace("&quot;", "\"")  # &quot; → " 로 변환
    return text.strip()  # 앞뒤 공백 제거

df_filtered_all_comp["description"] = df_filtered_all_comp["description"].apply(clean_description)


# ✅ 뉴스 본문에서 `description`과 유사도가 60% 이상인 문장 찾기
def extract_context_sentences(text, description, similarity_threshold=30):
    # `description` 정리
    description = clean_description(description)

    # 문장 분리
    sentences = re.split(r'(?<=\.)\s+|\n+', text)  # 문장 단위 분리 (더 자연스럽게)

    # 가장 유사한 문장 찾기
    matched_idx = None
    max_similarity = 0

    for idx, sent in enumerate(sentences):
        similarity = fuzz.ratio(description, sent)  # 유사도 계산 (0~100)
        if similarity >= similarity_threshold and similarity > max_similarity:
            max_similarity = similarity
            matched_idx = idx

    # 앞, 뒤 문장과 함께 선택하여 `imp_sent` 생성
    if matched_idx is not None:
        start_idx = max(0, matched_idx - 1)  # 앞 문장이 있으면 가져오기
        end_idx = min(len(sentences) - 1, matched_idx + 1)  # 뒤 문장이 있으면 가져오기
        imp_sent = " ".join(sentences[start_idx:end_idx + 1])  # 앞, 현재, 뒤 문장 합치기
    else:
        imp_sent = "관련 문장 없음"  # 매칭된 문장이 없을 경우 예외 처리

    return imp_sent

# ✅ newspaper3k를 사용하여 뉴스 제목과 본문 크롤링 후 `imp_sent` 생성
def extract_news_data(row):
    try:
        article = Article(row['org_link'], language='ko')
        article.download()
        article.parse()

        title = article.title.strip()  # 기사 제목 추출
        cleaned_description = clean_description(row['description'])  # `description` 정리
        imp_sent = extract_context_sentences(article.text, cleaned_description)  # `description` 기반 중요 문장 추출
        if imp_sent == "관련 문장 없음":
          imp_sent = cleaned_description

        return title, imp_sent
    except Exception as e:
        print(f"❌ {row['org_link']} 크롤링 오류: {e}")
        return "제목 없음", "본문 없음"

# ✅ 데이터프레임에 `new_title`과 `imp_sent` 컬럼 추가
df_filtered_all_comp[['new_title', 'imp_sent']] = df_filtered_all_comp.apply(lambda row: pd.Series(extract_news_data(row)), axis=1)

# ✅ 최종 데이터 출력
display(df_filtered_all_comp)


❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음
❌ 뉴스 없음 크롤링 오류: Article `download()` failed with No connection adapters were found for ':/뉴스 없음' on URL :/뉴스 없음


,cnt,company,title,description,org_link,link,pDate,new_title,imp_sent
0,1,정보보안,"동서발전, ‘<b>정보</b>보호 국제 표준인증’ 10년 연속 유지",정보보호인증(ISO27001)은 정보보호 및 관리 분야의 가장 권위 있는 국제인증으...,http://www.lawissue.co.kr/view.php?ud=20250203...,http://www.lawissue.co.kr/view.php?ud=20250203...,20250203,"동서발전, ‘정보보호 국제 표준인증’ 10년 연속 유지",[로이슈 전여송 기자]한국동서발전(사장 권명호)는 국제표준화기구(ISO)가 제정한 ...
1,1,클라우드,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203,제목 없음,본문 없음
2,1,제로트러스트,"[기업INSIDE] SGA솔루션즈, <b>제로트러스트</b>로 차세대 보안 선도",제로트러스트 보안을 앞세워 주목받는 국내 보안기업 SGA솔루션를 김경문 기자가 취재...,https://news.mtn.co.kr/news-detail/20250203164...,https://news.mtn.co.kr/news-detail/20250203164...,20250203,"[기업INSIDE] SGA솔루션즈, 제로트러스트로 차세대 보안 선도",이에 보안업계가 새로운 전환점을 맞이하고 있는데요. 제로트러스트 보안을 앞세워 주목...
3,1,정보유출사고,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203,제목 없음,본문 없음
4,1,나이스,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203,제목 없음,본문 없음
5,1,블록체인,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203,제목 없음,본문 없음
6,1,STO,"[마켓인]부동산 조각투자 스타트업 원컵, 싱가포르 진출 본격화",조각투자 플랫폼 원컵프로 운영사 원컵이 싱가포르 RWA(실물연계자산) 토큰 발행을 ...,http://www.edaily.co.kr/news/newspath.asp?news...,https://n.news.naver.com/mnews/article/018/000...,20250203,"[마켓인]부동산 조각투자 스타트업 원컵, 싱가포르 진출 본격화",[이데일리 마켓in 김연서 기자] 조각투자 플랫폼 원컵프로 운영사 원컵이 싱가포르 ...
7,1,AI,뉴스 없음,뉴스 없음,뉴스 없음,뉴스 없음,20250203,제목 없음,본문 없음
8,7,망분리,[주간 클라우드 동향/2월①] MS 성장엔진 클라우드 4분기 실적에 쏟아진...,◆국가망보안체계 예고편 나왔다…'N²SF 가이드라인' 초안 공개=공공기관이 망분리 ...,https://www.ddaily.co.kr/page/view/20250203094...,https://n.news.naver.com/mnews/article/138/000...,20250203,[주간 클라우드 동향/2월①] MS 성장엔진 클라우드 4분기 실적에 쏟아진 우려,더레지스터에 따르면 VM웨어는 VCF를 제외한 모든 제품에 3년 선납 약정을 강요하...
9,2,IT업계,"[뉴공 아카이브]박태웅 녹서포럼 의장, 유시민 작가: 美 <b>IT업계</b> 뒤흔...","아래 인터뷰는 2025년 2월 03일자, ""김어준의 겸손은힘들다 뉴스공장"" 방송분입...",http://www.ddanzi.com/834253806,http://www.ddanzi.com/834253806,20250203,"[뉴공 아카이브]박태웅 녹서포럼 의장, 유시민 작가: 美 IT업계 뒤흔든 ’딥시크 ...","아래 인터뷰는 2025년 2월 03일자, ""김어준의 겸손은힘들다 뉴스공장"" 방송분입..."


In [40]:
df = df_filtered_all_comp[['pDate', 'company', 'new_title', 'org_link', 'imp_sent']]

In [41]:
df

,pDate,company,new_title,org_link,imp_sent
0,20250203,정보보안,"동서발전, ‘정보보호 국제 표준인증’ 10년 연속 유지",http://www.lawissue.co.kr/view.php?ud=20250203...,[로이슈 전여송 기자]한국동서발전(사장 권명호)는 국제표준화기구(ISO)가 제정한 ...
1,20250203,클라우드,제목 없음,뉴스 없음,본문 없음
2,20250203,제로트러스트,"[기업INSIDE] SGA솔루션즈, 제로트러스트로 차세대 보안 선도",https://news.mtn.co.kr/news-detail/20250203164...,이에 보안업계가 새로운 전환점을 맞이하고 있는데요. 제로트러스트 보안을 앞세워 주목...
3,20250203,정보유출사고,제목 없음,뉴스 없음,본문 없음
4,20250203,나이스,제목 없음,뉴스 없음,본문 없음
5,20250203,블록체인,제목 없음,뉴스 없음,본문 없음
6,20250203,STO,"[마켓인]부동산 조각투자 스타트업 원컵, 싱가포르 진출 본격화",http://www.edaily.co.kr/news/newspath.asp?news...,[이데일리 마켓in 김연서 기자] 조각투자 플랫폼 원컵프로 운영사 원컵이 싱가포르 ...
7,20250203,AI,제목 없음,뉴스 없음,본문 없음
8,20250203,망분리,[주간 클라우드 동향/2월①] MS 성장엔진 클라우드 4분기 실적에 쏟아진 우려,https://www.ddaily.co.kr/page/view/20250203094...,더레지스터에 따르면 VM웨어는 VCF를 제외한 모든 제품에 3년 선납 약정을 강요하...
9,20250203,IT업계,"[뉴공 아카이브]박태웅 녹서포럼 의장, 유시민 작가: 美 IT업계 뒤흔든 ’딥시크 ...",http://www.ddanzi.com/834253806,"아래 인터뷰는 2025년 2월 03일자, ""김어준의 겸손은힘들다 뉴스공장"" 방송분입..."


In [42]:
# import pandas as pd
# from openpyxl import load_workbook
# from openpyxl.styles import Font

# # 기존 엑셀 파일 경로
# file_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"
# output_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"

# # 엑셀 파일 불러오기
# wb = load_workbook(file_path)

# # 작업용2 시트 가져오기 (없으면 생성)
# if "업계현황" in wb.sheetnames:
#     ws = wb["업계현황"]
# else:
#     ws = wb.create_sheet("업계현황")

# # 데이터 추가 시작 셀
# start_row = 3  # 데이터 시작 행 번호
# start_col = 2  # 데이터 시작 열 번호

# # 현재 입력할 행 번호
# current_row = start_row

# # 데이터 추가
# for row_idx, row in df.iterrows():
#     excel_row_number = current_row


#     # 공시 제목이 '공시 없음'인 경우 스킵
#     if row['new_title'] == '제목 없음':
#         current_row += 1
#         continue

#     # 공시 제목 셀 추가 및 하이퍼링크 처리
#     title_cell = ws.cell(row=current_row, column=start_col, value=row['new_title'])
#     title_cell.hyperlink = row['org_link']
#     title_cell.font = Font(color="0000FF", underline="single")
#     sentence_cell = ws.cell(row=current_row, column=start_col+1, value=row['imp_sent'])

#     # 다음 행으로 이동
#     current_row += 1

# # 엑셀 파일 저장
# wb.save(output_path)
# print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")

In [43]:
# import pandas as pd
# from openpyxl import load_workbook
# from openpyxl.styles import Font

# # 기존 엑셀 파일 경로
# file_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"
# output_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"

# # 엑셀 파일 불러오기
# wb = load_workbook(file_path)

# # 작업용 시트 가져오기 (없으면 생성)
# if "업계현황" in wb.sheetnames:
#     ws = wb["업계현황"]
# else:
#     ws = wb.create_sheet("업계현황")

# # 데이터 추가 시작 위치
# start_row = 3  # 데이터 시작 행 번호
# start_col = 2  # 데이터 시작 열 번호

# # 현재 입력할 행 번호
# current_row = start_row

# # ✅ 병합된 셀의 첫 번째 좌표를 찾는 함수
# def get_top_left_cell(merged_ranges, row, col):
#     """ 병합된 셀 내의 첫 번째(좌상단) 셀 주소 반환 """
#     for merged_range in merged_ranges:
#         min_col, min_row, max_col, max_row = merged_range.bounds
#         if min_row <= row <= max_row and min_col <= col <= max_col:
#             return ws.cell(row=min_row, column=min_col)  # 병합된 첫 번째 셀 반환
#     return ws.cell(row=row, column=col)  # 병합되지 않은 경우 원래 셀 반환

# # ✅ 엑셀에서 병합된 셀 정보 가져오기
# merged_ranges = list(ws.merged_cells.ranges)

# # ✅ 데이터 추가
# for row_idx, row in df.iterrows():
#     excel_row_number = current_row

#     # 공시 제목이 '제목 없음'인 경우 스킵
#     if row['new_title'] == '제목 없음':
#         current_row += 1
#         continue

#     # ✅ 병합된 경우, 첫 번째 셀을 찾아서 값 입력
#     title_cell = get_top_left_cell(merged_ranges, current_row, start_col)
#     title_cell.value = row['new_title']
#     title_cell.hyperlink = row['org_link']
#     title_cell.font = Font(color="0000FF", underline="single")

#     # ✅ 'imp_sent'도 병합된 경우 첫 번째 셀에 입력
#     sentence_cell = get_top_left_cell(merged_ranges, current_row, start_col + 1)
#     sentence_cell.value = row['imp_sent']

#     # 다음 행으로 이동
#     current_row += 1

# # ✅ 엑셀 파일 저장
# wb.save(output_path)
# print(f"✅ 데이터가 {output_path} 파일에 성공적으로 추가되었습니다.")


In [44]:
# import pandas as pd
# from openpyxl import load_workbook
# from openpyxl.styles import Font

# # 기존 엑셀 파일 경로
# file_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"
# output_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"

# # 엑셀 파일 불러오기
# wb = load_workbook(file_path)

# # 작업용 시트 가져오기 (없으면 생성)
# if "업계현황" in wb.sheetnames:
#     ws = wb["업계현황"]
# else:
#     ws = wb.create_sheet("업계현황")

# # 데이터 추가 시작 위치
# start_row = 3  # 데이터 시작 행 번호
# start_col = 2  # 데이터 시작 열 번호

# # 현재 입력할 행 번호
# current_row = start_row

# # ✅ 병합된 셀의 첫 번째 좌표 및 병합 크기 가져오는 함수
# def get_top_left_cell_and_span(merged_ranges, row, col):
#     """ 병합된 셀의 첫 번째 좌표와 병합된 행 개수를 반환 """
#     for merged_range in merged_ranges:
#         min_col, min_row, max_col, max_row = merged_range.bounds
#         if min_row <= row <= max_row and min_col <= col <= max_col:
#             row_span = max_row - min_row + 1  # 병합된 행 개수
#             return ws.cell(row=min_row, column=min_col), row_span  # 병합된 첫 번째 셀과 높이 반환
#     return ws.cell(row=row, column=col), 1  # 병합되지 않은 경우 높이 1 반환

# # ✅ 엑셀에서 병합된 셀 정보 가져오기
# merged_ranges = list(ws.merged_cells.ranges)

# # ✅ 데이터 추가
# for row_idx, row in df.iterrows():
#     excel_row_number = current_row

#     # 공시 제목이 '제목 없음'인 경우 스킵
#     # if row['new_title'] == '제목 없음':
#     #   title_cell, row_span = get_top_left_cell_and_span(merged_ranges, current_row, start_col)
#     #   current_row += row_span
#     #   continue

#     # ✅ 병합된 경우, 첫 번째 셀을 찾아서 값 입력 + 병합된 크기만큼 건너뛰기
#     title_cell, row_span = get_top_left_cell_and_span(merged_ranges, current_row, start_col)
#     if row['new_title'] == '제목 없음':
#       current_row += row_span + 1
#       continue
#     title_cell.value = row['new_title']
#     title_cell.hyperlink = row['org_link']
#     title_cell.font = Font(color="0000FF", underline="single")

#     # ✅ 'imp_sent'도 병합된 경우 첫 번째 셀에 입력
#     sentence_cell, _ = get_top_left_cell_and_span(merged_ranges, current_row, start_col + 1)
#     sentence_cell.value = row['imp_sent']

#     # ✅ 병합된 셀 크기만큼 `current_row`를 증가 (다음 데이터 위치 조정)
#     current_row += row_span + 1

# # ✅ 엑셀 파일 저장
# wb.save(output_path)
# print(f"✅ 데이터가 {output_path} 파일에 성공적으로 추가되었습니다.")


In [45]:
# import pandas as pd
# from openpyxl import load_workbook

# # 엑셀 파일 경로
# excel_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"
# new_excel_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"  # 새 파일 경로
# # sheet_name = "24.3Q_2411"
# sheet_name = "업계현황"

# # 기존 엑셀 파일 불러오기 (서식 유지)
# wb = load_workbook(excel_path)
# ws_old = wb[sheet_name]

# # 시트 이름이 기존에 존재하는지 확인
# if sheet_name in wb.sheetnames:
#     ws = wb[sheet_name]  # 기존 시트를 선택
#     print(f"기존 시트 '{sheet_name}'에 데이터를 추가합니다.")
# else:
#     ws = wb.copy_worksheet(ws_old)  # 기존 시트를 복제
#     ws.title = sheet_name  # 복제한 시트의 이름 변경
#     print(f"새로운 시트 '{sheet_name}'을(를) 생성하였습니다.")

# # 데이터프레임 로드 (bs_info_df, CIS_info_df는 미리 정의되어 있어야 함)
# # bs_info_df
# # CIS_info_df

# # 엑셀의 5행과 21행에 있는 회사명 찾기
# target_rows = [3,4,5,6,7,8,9,10,11,12,13,14]  # openpyxl은 1-based index (pandas는 0-based index)
# for row in target_rows:
#   company_name = ws.cell(row=row, column=1).value  # 회사명 가져오기
#   title_cell = ws.cell(row=row , column=2, value= df[df['company'] == company_name]['new_title'].values[0])
#   title_cell.hyperlink = df[df['company'] == company_name]['org_link'].values[0]
#   sentence_cell = ws.cell(row=row, column=3, value= df[df['company'] == company_name]['imp_sent'].values[0])

# # 엑셀 파일 저장
# wb.save(new_excel_path)
# print(f"데이터프레임이 {output_path} 파일에 성공적으로 추가되었습니다.")


In [48]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, Alignment

# 엑셀 파일 경로
excel_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"
new_excel_path = "/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx"  # 새 파일 경로
sheet_name = "업계현황"
output_sheet_name = "업계현황"  # 새로운 시트 이름

# 데이터프레임 로드 (bs_info_df, CIS_info_df는 미리 정의되어 있어야 함)
# 예시로 df를 bs_info_df로 지정합니다. 필요에 따라 변경하세요.


# 기존 엑셀 파일 불러오기 (서식 유지)
wb = load_workbook(excel_path)
ws_old = wb[sheet_name]

# 시트 이름이 기존에 존재하는지 확인
if output_sheet_name in wb.sheetnames:
    ws = wb[output_sheet_name]  # 기존 시트를 선택
    print(f"기존 시트 '{output_sheet_name}'에 데이터를 추가합니다.")
else:
    ws = wb.copy_worksheet(ws_old)  # 기존 시트를 복제
    ws.title = output_sheet_name  # 복제한 시트의 이름 변경
    print(f"새로운 시트 '{output_sheet_name}'을(를) 생성하였습니다.")

# 엑셀의 3행부터 14행까지 있는 회사명 찾기
target_rows = range(3, 15)  # openpyxl은 1-based index (pandas는 0-based index)
for row in target_rows:
    company_name = ws.cell(row=row, column=1).value  # 회사명 가져오기
    if company_name in df['company'].values:
        # 'new_title' 값 가져오기
        title_value = df.loc[df['company'] == company_name, 'new_title'].values[0]
        if title_value == '제목 없음':
            print(f"'{company_name}'의 'new_title'이 '제목 없음'이므로 건너뜁니다.")
            continue  # 'new_title'이 '제목 없음'이면 해당 행 건너뛰기
        # 'new_title' 값을 2열에 추가
        title_cell = ws.cell(row=row, column=2, value=title_value)
        # 'org_link'를 하이퍼링크로 추가
        org_link_value = df.loc[df['company'] == company_name, 'org_link'].values[0]
        title_cell.hyperlink = org_link_value
        # 'imp_sent' 값을 3열에 추가
        imp_sent_value = df.loc[df['company'] == company_name, 'imp_sent'].values[0]
        sentence_cell = ws.cell(row=row, column=3, value=imp_sent_value)
        # 셀 서식 설정 (예: 글꼴 굵게, 가운데 정렬)
        title_cell.font = Font(bold=True)
        title_cell.alignment = Alignment(horizontal='center')
        sentence_cell.alignment = Alignment(wrap_text=True)
    else:
        print(f"데이터프레임에 '{company_name}'에 대한 정보가 없습니다.")

# 엑셀 파일 저장
wb.save(new_excel_path)
print(f"데이터프레임이 '{new_excel_path}' 파일에 성공적으로 추가되었습니다.")



기존 시트 '업계현황'에 데이터를 추가합니다.
'클라우드'의 'new_title'이 '제목 없음'이므로 건너뜁니다.
'정보유출사고'의 'new_title'이 '제목 없음'이므로 건너뜁니다.
데이터프레임에 '나이스(NEIS)'에 대한 정보가 없습니다.
'블록체인'의 'new_title'이 '제목 없음'이므로 건너뜁니다.
'AI'의 'new_title'이 '제목 없음'이므로 건너뜁니다.
데이터프레임에 '기타'에 대한 정보가 없습니다.
데이터프레임이 '/content/drive/MyDrive/webcrawling/엑셀/SGA업계 및 경쟁사 현황4.xlsx' 파일에 성공적으로 추가되었습니다.
